## Working with an LLM programmatically

You have certainly interacted before with a Large Language Model (LLM) like ChatGPT. This is usually done through a UI or an application.

In this Notebook, we are going to use Python to connect and query an LLM directly through its API. For this Lab we have selected the model **Mistral-7B Instruct v2**.(https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2). This is a fully Open Source model (Apache 2.0 license), that although much lighter than other commercial or open source models is very capable, especially with the tasks we intend to use it for.

This model has already been deployed on the Lab cluster because even if it's a smaller model, it still needs a GPU with 24GB of RAM to run...

### Requirements and Imports

If you have selected the right workbench image to launch as per the Lab's instructions, you should already have all the needed libraries. If not uncomment the first line in the next cell to install all the right packages. We will then import the libraries we need.

In [ ]:
from langchain.chains import LLMChain
from langchain_community.llms import VLLMOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

### Langchain

Langchain (https://www.langchain.com/) is a framework for developing applications powered by language models. It will take care for us of all the boilerplate code we would have to manually write to properly query an LLM.

We will start by creating an **llm** instance, defined by the location where the LLM API can be queried and some parameters that will be applied to the model. For example, `max_new_tokens` will instruct the model to answer with a maximum of 512 tokens (words or parts of words). `temperature`, set really low here, will instruct the model to stay truth-grounded, and not try to be too 'creative'. After all, we're not trying to write a fancy poem here!

In [ ]:
inference_server_url = 'REPLACE_ME'
token = ''
model_name = ''

We also need a **template** to be applied to every request we are sending to the model (the 'Prompt').

When querying a model, you almost never want to send directly what the user has typed. On top of this entry, you need to give proper instructions to the model so that it knows how to handle it: what and how to answer, what NOT to answer, the tone it must use...

Langchain allows us to now easily 'stitch' those elements together and create a **conversation** object that we will use to query the model.

In [ ]:
llm = VLLMOpenAI(
    openai_api_key='NONE',
    openai_api_base=f'{inference_server_url}/v1',
    model_name=model_name or '/mnt/models/',
    max_tokens=512,
    top_p=0.95,
    temperature=0.01,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    default_headers={'Authorization': f'Bearer {token}'},
)

template = '''<s>[INST]<<SYS>>
You are a helpful, respectful and honest assistant. Always be as helpful as possible, while being safe.
You will be asked a question, to which you must give an answer.
Your answer should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content.
Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct.
If you don't know the answer to a question, answer 'I don't know'.
<</SYS>>

### QUESTION:
{input}

### ANSWER:
[/INST]
'''
PROMPT = PromptTemplate(input_variables=['input'], template=template)

conversation = LLMChain(llm=llm, prompt=PROMPT, verbose=False)

query = 'What is Artificial Intelligence?'

conversation.predict(input=query);

## Basic HTTP queries

In [ ]:
from requests import get, post

response = get(f'{inference_server_url}/v1/models')
try:
    print(response.json())
except Exception:
    print(response.text)

In [ ]:
payload = {
    'messages': [{'content': 'Hi', 'role': 'user'}],
    'model': model_name,
}

response = post(f'{inference_server_url}/v1/chat/completions', json=payload)
try:
    print(response.json())
except Exception:
    print(response.text)